In [178]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
import tensorflow as tf

from sklearn.impute import KNNImputer


In [573]:
import pandas as pd

filename = 'data/natural_rocks.csv'
# Load your data into a DataFrame, assuming it's a CSV for this example
df = pd.read_csv(filename, encoding='iso-8859-1')


df.head()



/tmp/ipykernel_3685106/1818057510.py:5: DtypeWarning:

Columns (1,6,8,10,11,13,14,15,16,17,18,22,23,41,42,43,44,56,181,182,184,185,186,187,188,189,190,191) have mixed types. Specify dtype option on import or set low_memory=False.



,sample_id,sample_name,latitude,longitude,loc_prec,qgis_geom,datum,depth,material,rock_name,...,rock_facies,country,method,author,title,journal,year,doi,bibtex,data_source
0,1,93286771,-53.100000,73.800000,1000.0,0101000020e61000003333333333735240cdcccccccc8c...,unknown,NaN,NaN,dacite,...,NaN,tf,NaN,"champion, d. c., budd, a. r., hazell, m. s., a...",NaN,NaN,NaN,NaN,NaN,ozchem
1,2,93286774,-53.150000,73.670000,1000.0,0101000020e61000007b14ae47e16a5240333333333393...,unknown,NaN,NaN,phonolite,...,NaN,tf,NaN,"champion, d. c., budd, a. r., hazell, m. s., a...",NaN,NaN,NaN,NaN,NaN,ozchem
2,3,93286772,-53.080000,73.670000,1000.0,0101000020e61000007b14ae47e16a52400ad7a3703d8a...,unknown,NaN,NaN,phonolite,...,NaN,tf,NaN,"champion, d. c., budd, a. r., hazell, m. s., a...",NaN,NaN,NaN,NaN,NaN,ozchem
3,4,93286773,-53.150000,73.670000,1000.0,0101000020e61000007b14ae47e16a5240333333333393...,unknown,NaN,NaN,phonolite,...,NaN,tf,NaN,"champion, d. c., budd, a. r., hazell, m. s., a...",NaN,NaN,NaN,NaN,NaN,ozchem
4,5,gds158a,33.421055,130.398294,NaN,0101000020e61000002d060fd3be4c60409d685721e5b5...,wgs84,NaN,NaN,marble,...,NaN,jp,xrf,"yuhara, m.; shoji, y.; ishihara, y.; uto, c.",geochemical map of 26 elements in the gokayama...,fukuoka univ. sci. rep.,2010,NaN,NaN,dodai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022087,1022088,kh18-3,-72.500000,11.500000,NaN,0101000020e61000000000000000002740000000000020...,NaN,NaN,NaN,charnockite,...,granulite,aq,NaN,"Mikhalsky, E. V.; Sheraton, J. W. & Hahne, K.",Charnockite composition in relation to the tec...,Gondwana Research,2006.0,10.1016/j.gr.2005.11.007,Mikhalsky_etal2006,NaN
1022088,1022089,kh18-1,-72.500000,11.500000,NaN,0101000020e61000000000000000002740000000000020...,NaN,NaN,NaN,charnockite,...,granulite,aq,NaN,"Mikhalsky, E. V.; Sheraton, J. W. & Hahne, K.",Charnockite composition in relation to the tec...,Gondwana Research,2006.0,10.1016/j.gr.2005.11.007,Mikhalsky_etal2006,NaN
1022089,1022090,kh41-7,-72.500000,11.500000,NaN,0101000020e61000000000000000002740000000000020...,NaN,NaN,NaN,charnockite,...,granulite,aq,NaN,"Mikhalsky, E. V.; Sheraton, J. W. & Hahne, K.",Charnockite composition in relation to the tec...,Gondwana Research,2006.0,10.1016/j.gr.2005.11.007,Mikhalsky_etal2006,NaN
1022090,1022091,41049-2,-72.500000,11.500000,NaN,0101000020e61000000000000000002740000000000020...,NaN,NaN,NaN,charnockite,...,granulite,aq,NaN,"Mikhalsky, E. V.; Sheraton, J. W. & Hahne, K.",Charnockite composition in relation to the tec...,Gondwana Research,2006.0,10.1016/j.gr.2005.11.007,Mikhalsky_etal2006,NaN


In [574]:
desired_columns=['SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'SO3', 'TiO2', 'MnO', 'K2O',
       'Na2O', 'LOI', 'S gravity', 'Amorphous content', 'd50', 'Ca(OH)2',
       'CaCO3', 'Water', 'K2SO4', 'KOH', 'curing temp', 'age',]

In [575]:
# Create a new DataFrame with the desired columns
new_df = pd.DataFrame()

for column in desired_columns:
    if column in df.columns:
        # Add the existing column to the new DataFrame
        new_df[column] = df[column]
    else:
        # Case insensitive check and add if exists
        matches = [col for col in df.columns if col.lower() == column.lower()]
        if matches:
            new_df[column] = df[matches[0]]
        else:
            # Create a column with NaN values if it doesn't exist
            new_df[column] = np.nan

# Display the head of the new DataFrame to verify
print(new_df.head())

    SiO2  Al2O3  Fe2O3   CaO   MgO  SO3  TiO2   MnO   K2O   Na2O  ...  \
0  66.72  13.90   1.06  4.22  1.22  NaN  0.41  0.07  0.57   5.48  ...   
1  54.36  19.75   1.61  1.43  0.62  NaN  0.50  0.12  5.49  11.29  ...   
2  54.65  19.51   1.74  1.43  0.61  NaN  0.51  0.14  5.16  11.32  ...   
3  54.61  19.72   1.66  1.64  0.86  NaN  0.59  0.12  5.40  10.88  ...   
4    NaN    NaN    NaN   NaN   NaN  NaN   NaN   NaN   NaN    NaN  ...   

   S gravity  Amorphous content  d50  Ca(OH)2  CaCO3  Water  K2SO4  KOH  \
0        NaN                NaN  NaN      NaN    NaN    NaN    NaN  NaN   
1        NaN                NaN  NaN      NaN    NaN    NaN    NaN  NaN   
2        NaN                NaN  NaN      NaN    NaN    NaN    NaN  NaN   
3        NaN                NaN  NaN      NaN    NaN    NaN    NaN  NaN   
4        NaN                NaN  NaN      NaN    NaN    NaN    NaN  NaN   

   curing temp  age  
0          NaN  NaN  
1          NaN  NaN  
2          NaN  NaN  
3          NaN  NaN  


In [576]:
new_df['S gravity']=df['density_model']/1000 
new_df['d50']=10


In [577]:

new_df['Ca(OH)2']=3
new_df['CaCO3']=0
new_df['Water']=3.6
new_df['K2SO4']=0
new_df['KOH']=0.151
new_df['curing temp']=50
new_df['age']=10


In [578]:
new_df = new_df.clip(lower=0)

In [580]:
new_df

,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,...,S gravity,Amorphous content,d50,Ca(OH)2,CaCO3,Water,K2SO4,KOH,curing temp,age
0,66.72,13.90,1.06,4.22,1.22,NaN,0.41,0.07,0.57,5.48,...,2.719912,NaN,10,3,0,3.6,0,0.151,50,10
1,54.36,19.75,1.61,1.43,0.62,NaN,0.50,0.12,5.49,11.29,...,2.674271,NaN,10,3,0,3.6,0,0.151,50,10
2,54.65,19.51,1.74,1.43,0.61,NaN,0.51,0.14,5.16,11.32,...,2.681069,NaN,10,3,0,3.6,0,0.151,50,10
3,54.61,19.72,1.66,1.64,0.86,NaN,0.59,0.12,5.40,10.88,...,2.677631,NaN,10,3,0,3.6,0,0.151,50,10
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10,3,0,3.6,0,0.151,50,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022087,65.77,16.05,1.65,2.95,0.27,NaN,0.55,0.08,5.63,3.30,...,2.742024,NaN,10,3,0,3.6,0,0.151,50,10
1022088,62.39,16.13,2.79,3.81,0.46,NaN,0.78,0.11,5.12,3.31,...,2.770994,NaN,10,3,0,3.6,0,0.151,50,10
1022089,60.71,13.89,4.29,4.40,0.41,NaN,1.09,0.20,4.37,2.83,...,2.833671,NaN,10,3,0,3.6,0,0.151,50,10
1022090,63.75,16.62,2.09,3.51,0.28,NaN,0.69,0.07,5.57,3.24,...,2.755071,NaN,10,3,0,3.6,0,0.151,50,10


In [581]:
for column in new_df.columns:
    new_df[column] = pd.to_numeric(new_df[column], errors='coerce')

In [582]:
input_mask = new_df.notnull().astype(int).values


In [583]:
input_mask[0]

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [584]:
df_r3 = pd.read_csv('R3.csv')
y = df_r3[['heat release', 'CH consumption', 'Bound water']]
df_r3.head()


,Material name,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,...,Ca(OH)2,CaCO3,Water,K2SO4,KOH,curing temp,age,heat release,CH consumption,Bound water
0,fly ash 1,55.70,23.71,3.75,10.50,1.00,0.25,0.71,NaN,0.81,...,3.0,0.0,3.6,0.0,0.151,50,10.0,NaN,60.478088,NaN
1,fly ash 2,41.62,21.30,8.14,17.13,2.84,1.59,1.28,NaN,1.49,...,3.0,0.0,3.6,0.0,0.151,50,10.0,NaN,67.649402,NaN
2,fly ash 3,35.48,18.36,5.27,20.38,4.09,2.73,1.11,NaN,0.81,...,3.0,0.0,3.6,0.0,0.151,50,10.0,NaN,62.071713,NaN
3,fly ash 4,56.76,20.51,6.21,9.74,2.59,0.46,1.17,NaN,1.35,...,3.0,0.0,3.6,0.0,0.151,50,10.0,NaN,55.697211,NaN
4,ground granulated blast furnace slag 1,32.59,13.94,0.78,42.22,4.96,2.02,0.58,NaN,0.37,...,3.0,0.0,3.6,0.0,0.151,50,10.0,NaN,23.027888,NaN


In [585]:

df_r3 = df_r3.drop(columns=['Material name','heat release','CH consumption', 'Bound water'])

for column in df_r3.columns:
    df_r3[column] = pd.to_numeric(df_r3[column], errors='coerce')
    
new_df = pd.concat([new_df, df_r3], ignore_index=True)
new_df = new_df.clip(lower=0)

In [587]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def lightgbm_impute(data, target_column, features):
    # Split data into training (known target values) and testing (unknown target values)
    known = data[data[target_column].notna()]
    unknown = data[data[target_column].isna()]

    # If no missing values are present, nothing needs to be done
    if unknown.shape[0] == 0:
        return data

    # Prepare input features and target
    X_train = known[features].values
    y_train = known[target_column].values
    X_test = unknown[features].values

    # Determine if the target column is categorical
    if issubclass(data[target_column].dtype.type, np.integer):
        model = lgb.LGBMClassifier()  # Use classifier for categorical data
    else:
        model = lgb.LGBMRegressor()  # Use regressor for numerical data

    # Train the model
    model.fit(X_train, y_train)

    # Predict the missing values
    predicted_values = model.predict(X_test)

    # Fill in the missing values in the original dataframe
    data.loc[data[target_column].isna(), target_column] = predicted_values

    return data

# Example usage:
# Assuming 'new_df' is your DataFrame which contains missing values in several columns
columns = new_df.columns.tolist()


# Impute each column
for column in columns:
    if new_df[column].isna().any():
        imputed_df = lightgbm_impute(new_df, column, [col for col in columns if col != column])


# Print the imputed DataFrame to verify
print(imputed_df)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006908 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3093
[LightGBM] [Info] Number of data points in the train set: 667090, number of used features: 20
[LightGBM] [Info] Start training from score 55.834512
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006354 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3096
[LightGBM] [Info] Number of data points in the train set: 659994, number of used features: 20
[LightGBM] [Info] Start training from score 14.082636
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003578 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008133 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3430
[LightGBM] [Info] Number of data points in the train set: 1023737, number of used features: 20
[LightGBM] [Info] Start training from score 10.018378
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008196 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3427
[LightGBM] [Info] Number of data points in the train set: 1023873, number of used features: 20
[LightGBM] [Info] Start training from score 9.991782
              SiO2      Al2O3     Fe2O3        CaO       MgO       SO3  \
0        66.720000  13.900000  1.060000   4.220000  1.220000  6.383376   
1        54.360000  19.750000  1.61

In [588]:
number_of_rows_in_df = len(df)

# Remove the last rows corresponding to the length of df
imputed_df = imputed_df.iloc[:number_of_rows_in_df]


In [589]:
imputed_df

,SiO2,Al2O3,Fe2O3,CaO,MgO,SO3,TiO2,MnO,K2O,Na2O,...,S gravity,Amorphous content,d50,Ca(OH)2,CaCO3,Water,K2SO4,KOH,curing temp,age
0,66.720000,13.900000,1.06000,4.220000,1.220000,6.383376,0.410000,0.070000,0.570000,5.480000,...,2.719912,45.729164,10.0,3.0,0.0,3.6,0.0,0.151,50,10.0
1,54.360000,19.750000,1.61000,1.430000,0.620000,-0.110666,0.500000,0.120000,5.490000,11.290000,...,2.674271,50.063200,10.0,3.0,0.0,3.6,0.0,0.151,50,10.0
2,54.650000,19.510000,1.74000,1.430000,0.610000,0.011635,0.510000,0.140000,5.160000,11.320000,...,2.681069,47.444894,10.0,3.0,0.0,3.6,0.0,0.151,50,10.0
3,54.610000,19.720000,1.66000,1.640000,0.860000,0.557182,0.590000,0.120000,5.400000,10.880000,...,2.677631,51.882089,10.0,3.0,0.0,3.6,0.0,0.151,50,10.0
4,55.589607,9.105307,3.90869,6.752052,4.059952,0.523167,1.051915,0.271058,0.934433,1.540521,...,2.919812,55.559931,10.0,3.0,0.0,3.6,0.0,0.151,50,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022087,65.770000,16.050000,1.65000,2.950000,0.270000,0.044353,0.550000,0.080000,5.630000,3.300000,...,2.742024,48.791023,10.0,3.0,0.0,3.6,0.0,0.151,50,10.0
1022088,62.390000,16.130000,2.79000,3.810000,0.460000,-0.023292,0.780000,0.110000,5.120000,3.310000,...,2.770994,48.293871,10.0,3.0,0.0,3.6,0.0,0.151,50,10.0
1022089,60.710000,13.890000,4.29000,4.400000,0.410000,0.020479,1.090000,0.200000,4.370000,2.830000,...,2.833671,43.046868,10.0,3.0,0.0,3.6,0.0,0.151,50,10.0
1022090,63.750000,16.620000,2.09000,3.510000,0.280000,0.345257,0.690000,0.070000,5.570000,3.240000,...,2.755071,47.616278,10.0,3.0,0.0,3.6,0.0,0.151,50,10.0


In [590]:
def custom_nan_mse(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.is_nan(y_true))
    mask = tf.cast(mask, dtype=tf.float32)
    
    y_true = tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_true), y_true)
    squared_difference = tf.square(y_true - y_pred) * mask
    
    # Add a small constant (epsilon) to avoid division by zero
    epsilon = 1e-7
    return tf.reduce_sum(squared_difference) / (tf.reduce_sum(mask) + epsilon)

def custom_nan_nrmse(y_true, y_pred):
    mask = tf.math.logical_not(tf.math.is_nan(y_true))
    mask = tf.cast(mask, dtype=tf.float32)
    
    y_true = tf.where(tf.math.is_nan(y_true), tf.zeros_like(y_true), y_true)
    squared_difference = tf.square(y_true - y_pred) * mask
    mse = tf.reduce_sum(squared_difference) / tf.reduce_sum(mask)
    rmse = tf.sqrt(mse)
    
    # Mask y_true values, replacing masked values with infinity and negative infinity
    # to ensure they don't affect the min and max calculations
    y_true_masked_min = tf.where(mask > 0, y_true, tf.fill(tf.shape(y_true), tf.float32.max))
    y_true_masked_max = tf.where(mask > 0, y_true, tf.fill(tf.shape(y_true), tf.float32.min))
    
    # Compute the min and max of the non-NaN y_true values
    y_true_min = tf.reduce_min(y_true_masked_min)
    y_true_max = tf.reduce_max(y_true_masked_max)
    
    y_range = y_true_max - y_true_min
    epsilon = 1e-7
    nrmse = rmse / (y_range + epsilon)
    
    return nrmse




In [591]:
import tensorflow as tf

final_model = tf.keras.models.load_model('90andupnewonesch.keras', custom_objects={'custom_nan_mse': custom_nan_mse,'custom_nan_nrmse': custom_nan_nrmse})


In [592]:
import pickle


mean_heat_release = y['heat release'].mean()
std_heat_release = y['heat release'].std()

mean_ch_consumption = y['CH consumption'].mean()
std_ch_consumption = y['CH consumption'].std()

mean_bound_water = y['Bound water'].mean()
std_bound_water = y['Bound water'].std()

In [593]:


scaler = StandardScaler()

X_imputed = df_imputed.drop(columns=['heat release', 'CH consumption', 'Bound water'])
X_scaled = scaler.fit_transform(X_imputed)

In [594]:
def unstandardize(values, mean, std):
    """Convert standardized values back to the original scale."""
    return values * std + mean

In [595]:
X_scaled = scaler.transform(imputed_df)

predictions = final_model.predict([X_scaled,input_mask] )

print(predictions)

31941/31941 [==============================] - 19s 591us/step
[array([[ 0.04219044],
       [-0.2555876 ],
       [-0.18415076],
       ...,
       [-0.46766204],
       [-0.3705246 ],
       [-0.48182297]], dtype=float32), array([[-0.2877838 ],
       [-0.92403615],
       [-0.8957956 ],
       ...,
       [-1.3186651 ],
       [-1.011136  ],
       [-1.2011883 ]], dtype=float32), array([[ 0.0303557 ],
       [-0.3767316 ],
       [-0.3814194 ],
       ...,
       [-0.50630075],
       [-0.4118119 ],
       [-0.5272409 ]], dtype=float32)]


In [596]:
# Unstandardize predictions
predictions_unstd = [
    unstandardize(predictions[0].flatten(), mean_heat_release, std_heat_release),
    unstandardize(predictions[1].flatten(), mean_ch_consumption, std_ch_consumption),
    unstandardize(predictions[2].flatten(), mean_bound_water, std_bound_water)
]

In [301]:
heat_release_arr = np.array(predictions_unstd[0])
ch_consumption_arr = np.array(predictions_unstd[1])
bound_water_arr = np.array(predictions_unstd[2])
